##### STA 220 Data & Web Technologies for Data Analysis

# Lecture 10 - 02/05/26, Scraping

### Announcements
- First homework assignment due tomorrow (02/06/26)
- For the task 1b): Please use the condition 'SUM(GS) IS NULL' and not 'SUM(GS) = 0'
- For the task 3b): Please define the win/loss ratio as __1.0__*total_wins/total_losses

### Today's topics
 - Web Scraping: 
     - Tornado Watch
     - WhereTheISS

### Ressources
 - [WhereTheISS](wheretheiss.at)
 - [Tornado Watch](https://www.tornadohq.com/)

## SUMMARY OF THE 9TH LECTURE

### `pd.read_html'

Use `pd.read_html` whenever you want to read a table from a webpage. It is by far the most convenient way to do so.

Always provide your User-Agent and slow down your requests!

In [8]:
# Example
import pandas as pd

headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}

tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_area', storage_options=headers)

In [9]:
tables[1] # et voila!

City  ST Land area       Water area         Total area         \
            City  ST     (mi2) (km2)      (mi2)   (km2)      (mi2)  (km2)   
0          Sitka  AK    2870.2  7434     1904.3  4932.0     4774.5  12366   
1         Juneau  AK    2702.9  7000      555.1  1438.0     3258.0   8438   
2       Wrangell  AK    2556.1  6620      915.0  2370.0     3471.1   8990   
3      Anchorage  AK    1706.8  4421      237.7   616.0     1944.5   5036   
4    Tribune[a]*  KS     778.2  2016        0.0     0.0      778.2   2016   
..           ...  ..       ...   ...        ...     ...        ...    ...   
145       Toledo  OH      80.5   208        3.3     8.5       83.8    217   
146    Jonesboro  AR      80.2   208        0.6     1.6       80.7    209   
147      El Reno  OK      79.6   206        0.6     1.6       80.2    208   
148    Ellsworth  ME      79.3   205       14.6    38.0       93.9    243   
149      Caribou  ME      79.3   205        0.8     2.1       80.1    207   

    Population (2020)  
    Population (2020)  
0                8458  
1               32255  
2                2127  
3              291247  
4                1182  
..                ...  
145            270871  
146             78576  
147             16989  
148              8399  
149              7396  

[150 rows x 9 columns]

### Scrape manually (XPath/BS4)

In [3]:
import requests
import lxml.html as lx
import requests_cache
import time
from re import sub
session = requests_cache.CachedSession('../output/lecture9')

We want to create this seasonality chart from [here](https://foodwise.org/eat-seasonally/seasonality-charts/seasonality-chart-vegetables/).

For this, we need to take two steps:
- Get all vegetables listed on the page
- For each product, get the months where the product is 'in Season'

#### FIRST STEP

In [10]:
def get_products(page):
    time.sleep(0.5)
    url = 'https://foodwise.org/eat-seasonally/seasonality-charts/'
    response = session.get(url, headers = headers, params = {
        '_food_type': 'vegetable',
        '_paged': page
    })
    response.raise_for_status()
    html = lx.fromstring(response.text) # Parse the HTML
    return(html.xpath('//a[@class="card-image-title__outer-link"]/@href'))

In [11]:
all_veggies = [el for p in range(1,5) for el in get_products(p)]

In [6]:
all_veggies

['https://foodwise.org/foods/artichokes/',
 'https://foodwise.org/foods/arugula/',
 'https://foodwise.org/foods/asparagus/',
 'https://foodwise.org/foods/beets/',
 'https://foodwise.org/foods/bitter-melon/',
 'https://foodwise.org/foods/bok-choy/',
 'https://foodwise.org/foods/broccoli/',
 'https://foodwise.org/foods/broccoli-rabe/',
 'https://foodwise.org/foods/brussels-sprouts/',
 'https://foodwise.org/foods/burdock/',
 'https://foodwise.org/foods/cabbage/',
 'https://foodwise.org/foods/cactus-pads/',
 'https://foodwise.org/foods/cardoons/',
 'https://foodwise.org/foods/carrots/',
 'https://foodwise.org/foods/cauliflower/',
 'https://foodwise.org/foods/celeriac/',
 'https://foodwise.org/foods/celery/',
 'https://foodwise.org/foods/celtuce/',
 'https://foodwise.org/foods/chard/',
 'https://foodwise.org/foods/chickweed/',
 'https://foodwise.org/foods/chicory/',
 'https://foodwise.org/foods/collard-greens/',
 'https://foodwise.org/foods/corn/',
 'https://foodwise.org/foods/cress/',
 'ht

##### SECOND STEP

In [12]:
def get_months(produce_link): 
    time.sleep(0.05)
    response = session.get(produce_link, headers = headers)
    try: response.raise_for_status()
    except requests.HTTPError:
        return [None, []] 
    else:
        html = lx.fromstring(response.text)
        try: 
            string = html.xpath('//section[@class="sidebar__section"][h2[contains(text(), "In Season")]]/text()')[1]
        except:
            return [None, []] 
        else:
            month = sub(r'(In Season)|\W', ' ', string).split() 
            name = html.xpath("//h1/text()")[0]
            return [name, month]

In [13]:
seasonality_info = [get_months(url) for url in all_veggies]

In [14]:
seasonality_info

[['Artichokes',
  ['March',
   'April',
   'May',
   'June',
   'September',
   'October',
   'November',
   'December']],
 ['Arugula',
  ['January',
   'February',
   'March',
   'April',
   'May',
   'June',
   'July',
   'August',
   'September',
   'October',
   'November',
   'December']],
 ['Asparagus', ['February', 'March', 'April', 'May', 'June']],
 ['Beets',
  ['January',
   'February',
   'March',
   'April',
   'May',
   'June',
   'July',
   'August',
   'September',
   'October',
   'November',
   'December']],
 ['Bitter melon',
  ['June', 'July', 'August', 'September', 'October', 'November']],
 ['Bok choy',
  ['January',
   'February',
   'March',
   'April',
   'May',
   'June',
   'July',
   'August',
   'September',
   'October',
   'November',
   'December']],
 ['Broccoli',
  ['January',
   'February',
   'March',
   'April',
   'May',
   'June',
   'July',
   'August',
   'September',
   'October',
   'November',
   'December']],
 ['Broccoli rabe',
  ['January',
   '

### END OF SUMMARY LECTURE 8

#### How to combine everything?

In [15]:
year = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 
        'October', 'November', 'December']

In [16]:
month = get_months('https://foodwise.org/foods/artichokes/')

In [17]:
month

['Artichokes',
 ['March',
  'April',
  'May',
  'June',
  'September',
  'October',
  'November',
  'December']]

In [18]:
[item in month for item in year]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [19]:
def assemble_row(produce_link): 
    name, months = get_months(produce_link)
    months = [item in months for item in year]
    months.insert(0, name)
    return months

In [20]:
assemble_row('https://foodwise.org/foods/artichokes/')

['Artichokes',
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True]

In [21]:
data = [assemble_row(i) for i in all_veggies] 
data

[['Artichokes',
  False,
  False,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True],
 ['Arugula',
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 ['Asparagus',
  False,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  False,
  False,
  False,
  False],
 ['Beets',
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 ['Bitter melon',
  False,
  False,
  False,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  False],
 ['Bok choy',
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 ['Broccoli',
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 ['Broccoli rabe',
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True],
 ['Brussels sprouts',
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  Fal

In [23]:
df = pd.DataFrame(data)
df.shape

(76, 13)

In [24]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Artichokes,False,False,True,True,True,True,False,False,True,True,True,True
1,Arugula,True,True,True,True,True,True,True,True,True,True,True,True
2,Asparagus,False,True,True,True,True,True,False,False,False,False,False,False
3,Beets,True,True,True,True,True,True,True,True,True,True,True,True
4,Bitter melon,False,False,False,False,False,True,True,True,True,True,True,False


In [25]:
columnames = year.copy()
columnames.insert(0, 'Produce')
df.columns = columnames

In [26]:
df.head()

,Produce,January,February,March,April,May,June,July,August,September,October,November,December
0,Artichokes,False,False,True,True,True,True,False,False,True,True,True,True
1,Arugula,True,True,True,True,True,True,True,True,True,True,True,True
2,Asparagus,False,True,True,True,True,True,False,False,False,False,False,False
3,Beets,True,True,True,True,True,True,True,True,True,True,True,True
4,Bitter melon,False,False,False,False,False,True,True,True,True,True,True,False


## Tornado Watch 

We are interested in scraping and plotting the locations of all tornado warnings in the last 48 hours. 

See the link <a href="https://www.tornadohq.com/">here<a>.

In [4]:
import requests
import lxml.html as lx
import time
import pandas as pd

In [5]:
session2 = requests_cache.CachedSession('../output/lecture9b')

In [6]:
result = session2.get('https://www.tornadohq.com/')
result.raise_for_status

<bound method Response.raise_for_status of <Response [200]>>

In [7]:
html = lx.fromstring(result.text) # Parse the HTML

In [8]:
warnings = html.xpath('//pre')
warnings

[<Element pre at 0x120a25e90>,
 <Element pre at 0x120a4be00>,
 <Element pre at 0x180698050>,
 <Element pre at 0x1804a3890>,
 <Element pre at 0x1804a3750>,
 <Element pre at 0x180500640>,
 <Element pre at 0x1804c71d0>,
 <Element pre at 0x1804c72f0>,
 <Element pre at 0x1804cd590>,
 <Element pre at 0x1804cd480>,
 <Element pre at 0x1804e4850>,
 <Element pre at 0x1804e5750>,
 <Element pre at 0x1804bb6b0>,
 <Element pre at 0x1804f92b0>,
 <Element pre at 0x1804f8c90>,
 <Element pre at 0x120a43520>,
 <Element pre at 0x120a43ee0>,
 <Element pre at 0x1209d6690>,
 <Element pre at 0x1209d6750>,
 <Element pre at 0x18058dff0>]

In [9]:
len(warnings)

20

In [ ]:
#for each of the Warnings we want to access the text of the warning
warning = warnings[0].text
warning

'####018001486####\nWFUS54 KHGX 032155\nTORHGX\nTXC291-032215-\n/O.NEW.KHGX.TO.W.0004.260203T2155Z-260203T2215Z/\n\nBULLETIN - EAS ACTIVATION REQUESTED\nTornado Warning\nNational Weather Service Houston/Galveston TX\n355 PM CST Tue Feb 3 2026\n\nThe National Weather Service in League City has issued a\n\n* Tornado Warning for...\n  Central Liberty County in southeastern Texas...\n\n* Until 415 PM CST.\n\n* At 354 PM CST, a severe thunderstorm capable of producing a tornado\n  was located 10 miles northeast of Plum Grove, or 10 miles southeast\n  of Cleveland, moving east at 20 mph.\n\n  HAZARD...Tornado.\n\n  SOURCE...Radar indicated rotation.\n\n  IMPACT...Flying debris will be dangerous to those caught without \n           shelter. Mobile homes will be damaged or destroyed. \n           Damage to roofs, windows, and vehicles will occur.  Tree \n           damage is likely.\n\n* This tornadic thunderstorm will remain over mainly rural areas of\n  central Liberty County.\n\nPRECAUTIONA

In [11]:
for w in warnings:
    print(w.text)
    print("\n\n-----THIS IS A NEW WARNING-----\n\n")

####018001486####
WFUS54 KHGX 032155
TORHGX
TXC291-032215-
/O.NEW.KHGX.TO.W.0004.260203T2155Z-260203T2215Z/

BULLETIN - EAS ACTIVATION REQUESTED
Tornado Warning
National Weather Service Houston/Galveston TX
355 PM CST Tue Feb 3 2026

The National Weather Service in League City has issued a

* Tornado Warning for...
  Central Liberty County in southeastern Texas...

* Until 415 PM CST.

* At 354 PM CST, a severe thunderstorm capable of producing a tornado
  was located 10 miles northeast of Plum Grove, or 10 miles southeast
  of Cleveland, moving east at 20 mph.

  HAZARD...Tornado.

  SOURCE...Radar indicated rotation.

  IMPACT...Flying debris will be dangerous to those caught without 
           shelter. Mobile homes will be damaged or destroyed. 
           Damage to roofs, windows, and vehicles will occur.  Tree 
           damage is likely.

* This tornadic thunderstorm will remain over mainly rural areas of
  central Liberty County.

PRECAUTIONARY/PREPAREDNESS ACTIONS...

TAKE CO

Lets match the latitude-longitude pair after `LAT...LON`. 

In [12]:
from re import findall

In [13]:
#find all the lat long data
findall('(?<=LAT\.{3}LON\s)(\d+\s\d+)', warning)

<>:2: SyntaxWarning: "\." is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\."? A raw string is also an option.
<>:2: SyntaxWarning: "\." is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\."? A raw string is also an option.
/var/folders/47/h_qs1w2n1p3bbhgw5ddnx8900000gn/T/ipykernel_9960/1093437200.py:2: SyntaxWarning: "\." is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\."? A raw string is also an option.
  findall('(?<=LAT\.{3}LON\s)(\d+\s\d+)', warning)


['3023 9500']

In [14]:
#Split up the lat and long
findall('(?<=LAT\.{3}LON\s)(\d+\s\d+)', warning)[0].split()
# (?<=...)	Positive Lookbehind.
# group consisting of ? optional character, LAT...LON followed by any whitespace. \d: any digit, at least one occurence, whitespace, \d any digit, at least one occurence

<>:2: SyntaxWarning: "\." is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\."? A raw string is also an option.
<>:2: SyntaxWarning: "\." is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\."? A raw string is also an option.
/var/folders/47/h_qs1w2n1p3bbhgw5ddnx8900000gn/T/ipykernel_9960/3093491975.py:2: SyntaxWarning: "\." is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\."? A raw string is also an option.
  findall('(?<=LAT\.{3}LON\s)(\d+\s\d+)', warning)[0].split()


['3023', '9500']

Rename the coordinates in readable format & make the table with all the lat long based on warnings

In [15]:
coord_list = [findall('(?<=LAT\.{3}LON\s)(\d+\s\d+)', warning.text)[0].split() for warning in warnings]

<>:1: SyntaxWarning: "\." is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\."? A raw string is also an option.
<>:1: SyntaxWarning: "\." is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\."? A raw string is also an option.
/var/folders/47/h_qs1w2n1p3bbhgw5ddnx8900000gn/T/ipykernel_9960/1738556140.py:1: SyntaxWarning: "\." is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\."? A raw string is also an option.
  coord_list = [findall('(?<=LAT\.{3}LON\s)(\d+\s\d+)', warning.text)[0].split() for warning in warnings]


In [16]:
coord = pd.DataFrame(coord_list)
coord.columns = ['N', 'W']
coord = coord.map(lambda x: float(x) / 100) # convert location in readable format
coord['W'] = -coord['W'] # longitude to west is negative
coord.head()

,N,W
0,30.23,-95.00
1,30.24,-95.03
2,30.24,-95.04
3,30.23,-95.10
4,30.49,-85.77


In [17]:
coord

,N,W
0,30.23,-95.00
1,30.24,-95.03
2,30.24,-95.04
3,30.23,-95.10
4,30.49,-85.77
5,31.44,-84.44
6,31.16,-84.92
7,31.43,-84.66
8,31.49,-85.03
9,31.39,-85.08


In [18]:
import plotly.express as px
import geopandas as gpd

fig = px.scatter_mapbox(coord,
                        lat='N',
                        lon='W',
                        zoom=2,
                        mapbox_style="open-street-map")
fig.show()


/var/folders/47/h_qs1w2n1p3bbhgw5ddnx8900000gn/T/ipykernel_9960/2765489165.py:4: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(coord,


## Summary 

- Scraping does not necessarily return the desired, make use of error handling 
- Make use of the advantages of devtools to see how the website is structured

## ISS and Satellite Data

Let's have a look on this great website:
https://wheretheiss.at/

It even provides a documented API! Find the documentation [here](https://wheretheiss.at/w/developer)!

#### Introduction

In [19]:
import requests
import requests_cache
import pandas as pd

headers = {
    'User - agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:142.0) Gecko/20100101 Firefox/142.0',
}

session = requests_cache.CachedSession('../output/ISS')

The documentation says it contains a lot of satellites plus the ISS.

So, let's first check all available satellites!

In [20]:
url = 'https://api.wheretheiss.at/v1/satellites'

response = session.get(url, headers = headers)
response.raise_for_status()
print(response.headers)

{'Date': 'Thu, 05 Feb 2026 22:49:42 GMT', 'Server': 'Apache/2.2.22 (Ubuntu)', 'X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Rate-Limit-Limit': '350', 'X-Rate-Limit-Remaining': '349', 'X-Rate-Limit-Interval': '5 minutes', 'Access-Control-Allow-Origin': '*', 'X-Apache-Time': 'D=12994', 'Cache-Control': 'max-age=0, no-cache', 'Content-Length': '27', 'Keep-Alive': 'timeout=15, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'application/json'}


In [21]:
pd.DataFrame(list(response.headers.items()))

,0,1
0,Date,"Thu, 05 Feb 2026 22:49:42 GMT"
1,Server,Apache/2.2.22 (Ubuntu)
2,X-Powered-By,PHP/5.3.10-1ubuntu3.26
3,X-Rate-Limit-Limit,350
4,X-Rate-Limit-Remaining,349
5,X-Rate-Limit-Interval,5 minutes
6,Access-Control-Allow-Origin,*
7,X-Apache-Time,D=12994
8,Cache-Control,"max-age=0, no-cache"
9,Content-Length,27


We can get the rate limits from the headers!

Now, lets check out all satellites:

In [22]:
all_satelites = print(response.json())

[{'name': 'iss', 'id': 25544}]


Only the ISS :(

In [23]:
iss_id = str(response.json()[0]['id'])

Get the location of the ISS:

In [24]:
import requests

url = 'https://api.wheretheiss.at/v1/satellites/' + iss_id

response = session.get(url, headers = headers)
response.raise_for_status()
print(response.headers)

{'Date': 'Thu, 05 Feb 2026 22:52:27 GMT', 'Server': 'Apache/2.2.22 (Ubuntu)', 'X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Rate-Limit-Limit': '350', 'X-Rate-Limit-Remaining': '348', 'X-Rate-Limit-Interval': '5 minutes', 'Access-Control-Allow-Origin': '*', 'X-Apache-Time': 'D=15112', 'Cache-Control': 'max-age=0, no-cache', 'Content-Length': '313', 'Keep-Alive': 'timeout=15, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'application/json'}


In [25]:
lim_remaining = response.headers["X-Rate-Limit-Remaining"]
print(f'Remaining requests within the next five minutes: {lim_remaining}')

Remaining requests within the next five minutes: 348


In [26]:
print(response) # shows the status code, not the content itself

<Response [200]>


Note: this is a cached response!

In [27]:
data = response.json()
print(data) # here we go!

{'name': 'iss', 'id': 25544, 'latitude': -3.6578414412322, 'longitude': 107.14576982926, 'altitude': 420.51587678976, 'velocity': 27580.164518173, 'visibility': 'daylight', 'footprint': 4510.0870794149, 'timestamp': 1770331947, 'daynum': 2461077.4530903, 'solar_lat': -15.694843351115, 'solar_lon': 200.38798380604, 'units': 'kilometers'}


In [28]:
import pandas as pd

tbl = pd.DataFrame(data, index=[0])

In [29]:
tbl.head() # looks good

,name,id,latitude,longitude,altitude,velocity,visibility,footprint,timestamp,daynum,solar_lat,solar_lon,units
0,iss,25544,-3.657841,107.14577,420.515877,27580.164518,daylight,4510.087079,1770331947,2.461077e+06,-15.694843,200.387984,kilometers


<p>Find the developer rate limits <a href="https://wheretheiss.at/w/developer">here</a>!</p>

<div class="alert alert-danger">
    We MUST NOT send more then one request per second!
</div>


#### Accessing the location

Let's access the ISS' location for a specific timestamp:

In [30]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timezone, timedelta
import warnings
import time

In [ ]:
def get_positions(timestamps):
    response = session.get('https://api.wheretheiss.at/v1/satellites/' + iss_id + '/positions?timestamps=' + str(timestamps))
    response.raise_for_status()
    lim_remaining = response.headers["X-Rate-Limit-Remaining"]
    if int(lim_remaining) < 10: # only slow down if we have indeed 
        warnings.warn("Too many requests!")
    time.sleep(1)
    return(response.json())

def is_lit(data):
    status = data["visibility"]
    return(status == "daylight")

#### Insertion: timedate formats
Dealing with different timedate formats

In [31]:
tz = 'America/Los_Angeles'
def dt2epoch(date):
    tme = np.datetime64(date)
    return(tme.astype(np.int64))

print(dt2epoch("now"))
print(dt2epoch("2025-02-05 15:00:00"))
print(dt2epoch("2025-02-05 15:00:00-0800"))

1770332360
1738767600
1738796400


/var/folders/47/h_qs1w2n1p3bbhgw5ddnx8900000gn/T/ipykernel_9960/3763540327.py:3: UserWarning:

no explicit representation of timezones available for np.datetime64



In [32]:
tz = 'America/Los_Angeles'
def epoch2dt(epoch):
    return(pd.to_datetime(epoch, unit = "s", utc=True).tz_convert(tz))

In [37]:
current_time = np.datetime64('now') # np.datetime64('2026-02-05 05:00:00-0800') # np.datetime64('now')
current_time

np.datetime64('2026-02-05T23:00:14')

In [38]:
dt2epoch(current_time)

np.int64(1770332414)

In [39]:
epoch2dt(dt2epoch(current_time))

Timestamp('2026-02-05 15:00:14-0800', tz='America/Los_Angeles')

#### DUSK/DAWN

We consider twilight to be everything 30 minutes before and after dawn or dusk. (This is a very rough proxy!)

https://www.timeanddate.com/sun/usa/davis

In [40]:
current_time

np.datetime64('2026-02-05T23:00:14')

In [41]:
!pip install astral

In [59]:
from astral import LocationInfo
from astral.sun import sun
import datetime as dt

tz = 'America/Los_Angeles'
timestamp_dt = epoch2dt(current_time)
davis = LocationInfo("Davis", "California", tz, 38.544907, -121.740517)
s = sun(davis.observer, date=timestamp_dt.date(), tzinfo=davis.timezone)
sun_times = pd.Series([s['dawn'], s['dusk']])

In [60]:
sun_times

0   2026-02-05 06:40:32.142089-08:00
1   2026-02-05 18:01:55.018521-08:00
dtype: datetime64[ns, America/Los_Angeles]

In [61]:
from astral import LocationInfo
from astral.sun import sun
import datetime as dt
import pandas as pd

tz = 'America/Los_Angeles'
davis = LocationInfo("Davis", "California", tz, 38.544907, -121.740517)

today = pd.Timestamp("02-10-2026") # pd.Timestamp("today")
print('Current time: ' + str(today))

twilight = []

for i in range(4):
    day = today + pd.Timedelta(i, "days")
    s = sun(davis.observer, date=day, tzinfo=davis.timezone)

    twilight += [(s['dawn'] - pd.Timedelta(75, "minutes") + pd.Timedelta(sec, "seconds")).timestamp() for sec in range(0,90*60,10)]
    twilight += [(s['dusk'] - pd.Timedelta(15, "minutes") + pd.Timedelta(sec, "seconds")).timestamp() for sec in range(0,90*60,10)]
    print('Dawn for today + ' + str(i) + " day(s): " + str(s['dawn']))
    print('Dusk for today + ' + str(i) + " day(s): " + str(s['dusk']))

Current time: 2026-02-10 00:00:00
Dawn for today + 0 day(s): 2026-02-10 06:35:34.361979-08:00
Dusk for today + 0 day(s): 2026-02-10 18:07:18.645880-08:00
Dawn for today + 1 day(s): 2026-02-11 06:34:30.466978-08:00
Dusk for today + 1 day(s): 2026-02-11 18:08:22.953305-08:00
Dawn for today + 2 day(s): 2026-02-12 06:33:25.195902-08:00
Dusk for today + 2 day(s): 2026-02-12 18:09:27.091324-08:00
Dawn for today + 3 day(s): 2026-02-13 06:32:18.579461-08:00
Dusk for today + 3 day(s): 2026-02-13 18:10:31.049700-08:00


In [62]:
print(twilight[0:10])
print(len(twilight))

[1770729634.361979, 1770729644.361979, 1770729654.361979, 1770729664.361979, 1770729674.361979, 1770729684.361979, 1770729694.361979, 1770729704.361979, 1770729714.361979, 1770729724.361979]
4320


#### GET DATA

In [ ]:
import tqdm, requests
pos = []

url = "https://api.wheretheiss.at/v1/satellites/25544/positions?timestamps="

def get_positions(timestamps):
    response = session.get(url + str(timestamps))
    response.raise_for_status()
    lim_remaining = response.headers["X-Rate-Limit-Remaining"]
    if int(lim_remaining) < 200:
        warnings.warn("Two many requests!")
        time.sleep(1)
    time.sleep(1)
    return(response.json())

for ind in tqdm.tqdm(range(0, len(twilight), 10)):
#    progress = np.round(100*ind/len(epoch_time),1)
    timestamps = twilight[ind:(ind+10)]
    ts_string = ",".join(map(str, timestamps))
    
    pos += get_positions(ts_string)
    # time.sleep(1)

print(pos)

##### SAVE / LOAD DATA

In [ ]:
import json

with open('../output/ISS2.json','w+') as file:
    json.dump(pos, file)

In [64]:
import json

with open('../output/ISS.json','r') as file:
    pos = json.load(file)

##### CONTINUE

In [65]:
data = pd.DataFrame(pos)
data = data[data.visibility == "daylight"]
perc = int(len(data)/len(pos)*100)

print(f'The ISS is in daylight in {perc}% of the cases.')
data = data.drop(['name', 'id', 'velocity', 'visibility', 'daynum', 'solar_lat', 'solar_lon', 'units'], axis = 1).set_index("timestamp")
print(data)

The ISS is in daylight in 63% of the cases.
             latitude  longitude    altitude    footprint
timestamp                                                
1770729634  47.148514 -12.370971  419.242567  4503.607972
1770729644  46.876801 -11.553655  419.216562  4503.475530
1770729654  46.598706 -10.745064  419.190047  4503.340487
1770729664  46.314351  -9.945218  419.163060  4503.203031
1770729674  46.023866  -9.154137  419.135637  4503.063353
...               ...        ...         ...          ...
1771038881 -51.118449 -81.587063  445.844060  4636.676396
1771038891 -51.005386 -80.625991  445.801851  4636.468957
1771038901 -50.883768 -79.669954  445.755278  4636.240059
1771038911 -50.753676 -78.719261  445.704360  4635.989795
1771038921 -50.615194 -77.774221  445.649121  4635.718268

[2761 rows x 4 columns]


In [67]:
!pip install geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [geopy]


In [68]:
from geopy.distance import geodesic

davis = (38.544907, -121.740517)

def calc_diff(coords):
    point = (coords["latitude"], coords["longitude"])
    return(geodesic(point, davis).km)

dist = data.apply(calc_diff, axis=1)
data["distance"] = dist
near = (dist < data["footprint"])
data["is_near"] = near
data.head()

,latitude,longitude,altitude,footprint,distance,is_near
timestamp,,,,,,
1770729634,47.148514,-12.370971,419.242567,4503.607972,8217.815997,False
1770729644,46.876801,-11.553655,419.216562,4503.475530,8284.817136,False
1770729654,46.598706,-10.745064,419.190047,4503.340487,8351.849125,False
1770729664,46.314351,-9.945218,419.163060,4503.203031,8418.911364,False
1770729674,46.023866,-9.154137,419.135637,4503.063353,8486.002463,False


In [69]:
tbl = data[data["is_near"]]
len(tbl)

218

In [70]:
tbl

,latitude,longitude,altitude,footprint,distance,is_near
timestamp,,,,,,
1770733734,12.221378,-141.849231,418.589794,4500.281995,3531.396415,True
1770733744,12.723232,-141.470709,418.513074,4499.890900,3462.639247,True
1770733754,13.224414,-141.090525,418.439289,4499.514725,3393.895676,True
1770733764,13.724895,-140.708605,418.368421,4499.153386,3325.166508,True
1770733774,14.224645,-140.324875,418.300451,4498.806794,3256.452630,True
...,...,...,...,...,...,...
1770991388,51.766918,-70.762591,419.477993,4504.806772,4170.636477,True
1770991398,51.743246,-69.762705,419.463826,4504.734642,4239.695269,True
1770991408,51.710365,-68.764110,419.448513,4504.656677,4308.753080,True


##### SKIP THIS

In [71]:
from astral import LocationInfo
from astral.sun import sun
import datetime as dt

def is_twilight(entry):
    tz = 'America/Los_Angeles'
    timestamp_dt = epoch2dt(entry["timestamp"]) # convert timestamp to epoch time
    davis = LocationInfo("Davis", "California", tz, 38.544907, -121.740517)
    s = sun(davis.observer, date=timestamp_dt.date(), tzinfo=davis.timezone) # providing the data of the timestamp
    sun_times = pd.Series([s['dawn'], s['dusk']]) # calculating the timestamp fo the dawn and dusk on this specific day
#    time_diffs = np.abs(timestamp_dt - sun_times)
#    min_diff = time_diffs.min()
    if (timestamp_dt > s['dawn'] - pd.Timedelta(minutes=90) and timestamp_dt < s['dawn'] + pd.Timedelta(minutes=20)) or (
            timestamp_dt > s['dusk'] - pd.Timedelta(minutes=20) and timestamp_dt < s['dusk'] + pd.Timedelta(minutes=90)):
        tw = True
    else:
        tw = False
    return tw
#    return min_diff < pd.Timedelta(minutes=70)

twl = data.reset_index().apply(is_twilight, axis=1)
sum(twl)

2761

In [72]:
data.shape

(2761, 6)

In [73]:
twl

0       True
1       True
2       True
3       True
4       True
        ... 
2756    True
2757    True
2758    True
2759    True
2760    True
Length: 2761, dtype: bool

Next, we should calculate the distance between Davis and the projection of the ISS on the earth.

Remember: we don't have to do everything by hand. For most applications, there is a Python package.

In [74]:
!pip install geopy

In [75]:
from geopy.distance import geodesic

davis = (38.544907, -121.740517)

def calc_diff(coords):
    point = (coords["latitude"], coords["longitude"])
    return(geodesic(point, davis).km)

dist = tbl.apply(calc_diff, axis=1)
dist.head()

timestamp
1770733734    3531.396415
1770733744    3462.639247
1770733754    3393.895676
1770733764    3325.166508
1770733774    3256.452630
dtype: float64

##### CONTINUE HERE

In [76]:
tbl.head()

,latitude,longitude,altitude,footprint,distance,is_near
timestamp,,,,,,
1770733734,12.221378,-141.849231,418.589794,4500.281995,3531.396415,True
1770733744,12.723232,-141.470709,418.513074,4499.890900,3462.639247,True
1770733754,13.224414,-141.090525,418.439289,4499.514725,3393.895676,True
1770733764,13.724895,-140.708605,418.368421,4499.153386,3325.166508,True
1770733774,14.224645,-140.324875,418.300451,4498.806794,3256.452630,True


In [77]:
amin = np.argmin(tbl[tbl.is_near]["distance"])
minimizer = tbl[tbl.is_near].iloc[amin]
print(minimizer)

latitude       40.604518
longitude     -123.92691
altitude      418.589926
footprint    4500.282668
distance      295.918532
is_near             True
Name: 1770907205, dtype: object


In [78]:
epoch2dt(minimizer.name)

Timestamp('2026-02-12 06:40:05-0800', tz='America/Los_Angeles')

Compare it with: https://www.astroviewer.net/iss/en/observation.php

#### BEARING

In [79]:
tz = 'America/Los_Angeles'
davis = [38.544907, -121.740517]

import math

def calculate_bearing(point):
    """
    Calculates the bearing in degrees between two geolocations.
    Latitudes and longitudes should be provided in decimal degrees.
    """
    lat1_rad = math.radians(davis[0])
    lon1_rad = math.radians(davis[1])
    lat2_rad = math.radians(point[0])
    lon2_rad = math.radians(point[1])

    delta_lon = lon2_rad - lon1_rad

    y = math.sin(delta_lon) * math.cos(lat2_rad)
    x = math.cos(lat1_rad) * math.sin(lat2_rad) - \
        math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(delta_lon)

    bearing_rad = math.atan2(y, x)
    bearing_deg = math.degrees(bearing_rad)
    
    # Normalize bearing to be within 0-360 degrees
    bearing_deg = (bearing_deg + 360) % 360
    return bearing_deg

def get_cardinal_direction(point):
    """
    Converts a bearing in degrees (0-360) to a cardinal direction.
    """
    directions = ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]
    # Adjust bearing to align with the center of each 45-degree sector
    # For example, N is centered at 0, NE at 45, E at 90, etc.
    bearing = calculate_bearing(point)
    index = round(bearing / 45) % 8
    return directions[index]

In [80]:
# Example Usage:
point = [38.8977, -77.0365] 

bearing = calculate_bearing(point)
direction = get_cardinal_direction(point)

print(f"The bearing from is {bearing:.2f} degrees.")
print(f"The cardinal direction is: {direction}")

The bearing from is 75.03 degrees.
The cardinal direction is: E


The example point was the White House. It is indeed eastern of Davis, see:
[Google Maps](https://www.google.com/maps/dir/University+of+California,+Shields+Avenue,+Davis,+Kalifornien/38.8977,-77.0365/@38.2386811,-109.987335,5z/data=!3m1!4b1!4m9!4m8!1m5!1m1!1s0x80ead37f7489fa3f:0xecbfbb24087e8334!2m2!1d-121.7617125!2d38.5382322!1m0!3e4?entry=ttu&g_ep=EgoyMDI1MDkxNy4wIKXMDSoASAFQAw%3D%3D)

In [81]:
point = list(minimizer[["latitude", "longitude"]])
print(calculate_bearing(point))
print(get_cardinal_direction(point))

321.40024085472936
NW


Compare it with: https://www.astroviewer.net/iss/en/observation.php!

Waaaaaiiiit. Couldn't we just scrape this site?

In [82]:
import requests
import requests_cache
import pandas as pd
import lxml.html as lx

In [83]:
url = 'https://www.astroviewer.net/iss/en/observation.php'
headers = {
    'User - agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:142.0) Gecko/20100101 Firefox/142.0',
}

In [84]:
response = requests.get(url)
response.raise_for_status()

In [85]:
html = lx.fromstring(response.text)

In [86]:
table = html.xpath('//table[contains(@class, "passDetails")]')[0]

In [87]:
rows = table.xpath('//tbody/tr')

In [88]:
cells = table.xpath('//tbody/tr/td')

In [89]:
[c.text for c in cells]

['...',
 '...',
 '...',
 '...',
 '...',
 '...',
 '...',
 '...',
 '...',
 '...',
 '...',
 None]

In [90]:
[c.text_content() for c in rows[0].xpath('//td')]

['...',
 '...',
 '...',
 '...',
 '...',
 '...',
 '...',
 '...',
 '...',
 '...',
 '...',
 '...',
 'i']

It conains only '...'. Have a look at the HTML content!

And how to choose Davis in the first place?

## --> SELENIUM BROWSER